# 4. Explore and Clean

## Ensure Environment is Configured

**Go [here](01_import_data.ipynb) if you are following along and have not configured the virtual environment and installed dependencies.**

In [33]:
# Jupyter magic
%run ../util/dependencies.py
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Check for Cleanliness
### Define db Interface Methods

In [43]:
# Lets make a simple function to print our queries from the db, using psycopg2 . . .
def vectra_view(my_query):
    # connect to db
    db_connection = psycopg2.connect(dbname = 'vectra',
                                    host='localhost',
                                    user='robb',
                                    password='thek3yisK#')
    cur = db_connection.cursor()

    # return results
    cur.execute(my_query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

    # close
    cur.close()
    db_connection.close()

def vectra_commit(my_query):
    # connect to db
    db_connection = psycopg2.connect(dbname = 'vectra',
                                     host='localhost',
                                     user='robb',
                                     password='thek3yisK#')
    cur = db_connection.cursor()

    # excecute
    cur.execute(my_query)
    db_connection.commit()

    # close
    cur.close()
    db_connection.close()

# Generalized, so should work for any table . . .
def vectra_insert_df(df, table_name, conflict_cols):
    """
    Insert a pandas DataFrame into a PostgreSQL table using psycopg2.
    Handles arbitrary columns and conflict keys.
    """

    # Connect
    conn = psycopg2.connect(
        dbname='vectra',
        host='localhost',
        user='robb',
        password='thek3yisK#'
    )
    cur = conn.cursor()

    # Column list
    cols = list(df.columns)
    col_names = ", ".join(cols)

    # Placeholder for VALUES (%s, %s, %s...)
    placeholders = ", ".join(["%s"] * len(cols))

    # Conflict target
    conflict_target = ", ".join(conflict_cols)

    # Build SET col = EXCLUDED.col for all non-conflict columns
    update_assignments = ", ".join(
        [f"{col} = EXCLUDED.{col}" for col in cols if col not in conflict_cols]
    )

    # Build final SQL
    query = f"""
        INSERT INTO {table_name} ({col_names})
        VALUES %s
        ON CONFLICT ({conflict_target}) DO UPDATE
        SET {update_assignments};
    """

    # Convert DataFrame to list of tuples
    values = [tuple(row) for row in df.to_numpy()]

    # Bulk insert
    execute_values(cur, query, values)

    conn.commit()
    cur.close()
    conn.close()

    print(f"Inserted/updated {len(df)} rows into {table_name}.")


Testing methods here . . .

In [40]:
# Start off simple by checking basic tables . . .
q_table_names = """
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
"""

q_ata = """
    SELECT * 
    FROM ata_chapter
    ORDER BY ata_chapter
"""

vectra_view(q_ata)

('11', None)
('14', None)
('21', None)
('22', None)
('23', None)
('24', None)
('25', None)
('26', None)
('27', None)
('28', None)
('29', None)
('30', None)
('31', None)
('32', None)
('33', None)
('34', None)
('35', None)
('36', None)
('38', None)
('49', None)
('51', None)
('52', None)
('53', None)
('54', None)
('55', None)
('56', None)
('57', None)
('72', None)
('73', None)
('75', None)
('76', None)
('77', None)
('78', None)
('79', None)
('80', None)


### ATA Chapters
Important to note is that although this entity may not be necessary for analysis of common faulty systems (as we can just derive the ata chapter codes from the JASC), I included this table to allow identifying system (not subsystem) trends with their system names as opposed to just the ATA chapter itself.

**As a personal example from my experience with C-130H's (as ATA Chapters match TO -Technical Order- maintenance documentation), this would help distinguish subsystems -26-10 (fire detection) from -26-20 (fire suppression) wihtin the -26 (fire protection) system.**

An immediate issue that stood out to me was that ATA chapters are not defined in the SDRs (beyond the codes themselves). I can fix this by updating records with the appropriate names based on how the FAA defines each of these chapters.

To do this, I scrapped the ATA chapter definitions from [Aerospace Unlimited](https://www.aerospaceunlimited.com/ata-chapters/) (since this was the cleanest table I could find of these definitions), converted the data to a csv, cleaned the data, sorted, and imported it [here](../data/02_csv/ata_chapters.csv).

Additionally, the new ata chapters are **exhaustive**, this is to say **new ata chapters encountered by the Service Difficulty Reporting System (SDRS) are addressed**.

**Note:**
The following ATA chapters were excluded do to not being included in the JASC codes themselves, as well as not being utilized in the SDRS.
| ATA Chapter | Description |
| --- | --- |
| 115 | Flight Simulator Systems |
| 116 | Flight Simulator Cuing System |
| 72R | Engine - Reciprocating |
| 72T | Engine - Turbine/Turboprop / Ducted Fan/Unducted Fan |

Now lets update the ata_chapter table . . .

In [ ]:
-- verify structure
SELECT *
FROM ata_chapter
LIMIT 10;

In [44]:
# check table right now
import pandas as pd
ata_table = pd.read_csv('../data/02_csv/ata_chapters.csv', index_col=False)
print(ata_table.columns) ## one str and one int type

# lets clean up . . .
ata_table.rename(columns={' Name': 'description', 'Chapter': 'ata_chapter'}, inplace=True) # match entity schema
ata_table['description'] = ata_table['description'].str.strip() # just found out, like excel TRIM()!!
print(ata_table)

vectra_insert_df(ata_table, 'ata_chapter', ['ata_chapter'])

Index(['Chapter', ' Name'], dtype='str')
    ata_chapter                              description
0             0                                  General
1             1                       Maintenance Policy
2             2                               Operations
3             3                                  Support
4             4                Airworthiness Limitations
..          ...                                      ...
80           95                   Crew Escape And Safety
81           96          Missiles - Drones And Telemetry
82           97                         Wiring Reporting
83           98  Meteorological And Atmospheric Research
84           99                Electronic Warfare System

[85 rows x 2 columns]
Inserted/updated 85 rows into ata_chapter.


In [ ]:
-- Verify through native SQL
SELECT *
FROM ata_chapter
ORDER BY ata_chapter ASC;

Getting a weird '14' ata_channel, lets see where this is coming from. I couldn't find it in the [ata_chapters.csv](../data/02_csv/ata_chapters.csv) so it must be from the sdr's themselves?

In [ ]:
vectra_view("""
    SELECT count(*)
    FROM ata_chapter
""")                    # -> 86 total chapters

ata_table.info()        # -> 85 total chapters

(86,)
<class 'pandas.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   ata_chapter  85 non-null     int64
 1   description  85 non-null     str  
dtypes: int64(1), str(1)
memory usage: 3.0 KB


Looking over the previous query, '14' has no description, but it IS detailed in the "[FEDERAL AVIATION ADMINISTRATION JOINT AIRCRAFT SYSTEM/COMPONENT CODE TABLE AND DEFINITIONS](https://sdrs.faa.gov/documents/JASC_Code.pdf)" as "Hardware".

In [51]:
-- Update 14: Hardware
UPDATE ata_chapter
SET description = 'Hardware'
WHERE ata_chapter = 14;

-- check
SELECT *
FROM ata_chapter
ORDER BY ata_chapter;

SyntaxError: invalid syntax (949731905.py, line 1)

### JASC
These codes are critical for identifying subsystems. To map the current JASC codes as well as the entire FAA JASC set (last updated Oct 2008), I scraped the information from the "[FEDERAL AVIATION ADMINISTRATION JOINT AIRCRAFT SYSTEM/COMPONENT CODE TABLE AND DEFINITIONS](https://sdrs.faa.gov/documents/JASC_Code.pdf)" and pasted it into [this text doc](../data/01_raw/jasc_codes_scrapped_from_faa.txt).

In [ ]:
SELECT *
FROM jasc_code
ORDER BY random()
LIMIT 10;

need descriptions again . . .

In [ ]:
# Import ../data/01_raw/jasc_codes_scrapped_from_faa.txt as dataframe
# NUMBER[SPACE][DESCRIPTION WITH LOTS OF COMMAS AND OTHER ANNOYING PUNCTUATION]

#open the file
with open('../data/01_raw/jasc_codes_scrapped_from_faa.txt') as f:
    lines = f.readLines()

jasc_table = pd.DataFrame()
jasc_table['jasc_code'] #STRING
jasc_table['ata_chapter'] #int
jasc_table['description'] #text